In [3]:
%load_ext autoreload
%autoreload 2

loading file
[Source](http://files.fast.ai/models/wt103/)

In [34]:
import os
import pickle
import datetime
import pandas as pd
import torch
import torch.nn as nn

from training_utils import training_loop, test_loop
from model_v2 import RNNLM
# from data_utils import IndexVectorizer, TextDataset, simple_tokenizer

## Downloading pre-trained weights

In [10]:
directory_pre_trained_models = '/scratch/olympus/projects/sentiment_analysis/machine_learnining/weights_pretrained'
os.makedirs(directory_pre_trained_models, exist_ok=True)

In [12]:
!wget http://files.fast.ai/models/wt103/fwd_wt103_enc.h5 -O $directory_pre_trained_models/fwd_wt103_enc.h5

--2018-12-04 17:14:17--  http://files.fast.ai/models/wt103/fwd_wt103_enc.h5
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 462387634 (441M) [text/plain]
Saving to: ‘/scratch/olympus/projects/sentiment_analysis/machine_learnining/weights_pretrained/fwd_wt103_enc.h5’

100%[======================================>] 462,387,634 19.2MB/s   in 22s    

2018-12-04 17:14:40 (19.6 MB/s) - ‘/scratch/olympus/projects/sentiment_analysis/machine_learnining/weights_pretrained/fwd_wt103_enc.h5’ saved [462387634/462387634]



In [13]:
!wget http://files.fast.ai/models/wt103/fwd_wt103.h5 -O $directory_pre_trained_models/fwd_wt103.h5

--2018-12-04 17:14:40--  http://files.fast.ai/models/wt103/fwd_wt103.h5
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 462387687 (441M) [text/plain]
Saving to: ‘/scratch/olympus/projects/sentiment_analysis/machine_learnining/weights_pretrained/fwd_wt103.h5’

100%[======================================>] 462,387,687 30.8MB/s   in 16s    

2018-12-04 17:14:56 (26.7 MB/s) - ‘/scratch/olympus/projects/sentiment_analysis/machine_learnining/weights_pretrained/fwd_wt103.h5’ saved [462387687/462387687]



In [14]:
!wget http://files.fast.ai/models/wt103/itos_wt103.pkl -O $directory_pre_trained_models/fitos_wt103.pkl

--2018-12-04 17:15:08--  http://files.fast.ai/models/wt103/itos_wt103.pkl
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4161252 (4.0M) [text/plain]
Saving to: ‘/scratch/olympus/projects/sentiment_analysis/machine_learnining/weights_pretrained/fitos_wt103.pkl’

100%[======================================>] 4,161,252   22.1MB/s   in 0.2s   

2018-12-04 17:15:08 (22.1 MB/s) - ‘/scratch/olympus/projects/sentiment_analysis/machine_learnining/weights_pretrained/fitos_wt103.pkl’ saved [4161252/4161252]



In [11]:
!ls -ltha $directory_pre_trained_models

total 886M
drwxrws---+ 2 ly501 smapp 4.0K Dec  4 17:15 .
drwxrws---+ 3 ly501 smapp 4.0K Dec  4 17:11 ..
-rwxrwx---+ 1 ly501 smapp 441M Mar 28  2018 fwd_wt103.h5
-rwxrwx---+ 1 ly501 smapp 441M Mar 28  2018 fwd_wt103_enc.h5
-rwxrwx---+ 1 ly501 smapp 4.0M Mar 28  2018 fitos_wt103.pkl


## Loading weights

In [12]:
encoder_file = os.path.join(directory_pre_trained_models, 'fwd_wt103.h5')
weights_file = os.path.join(directory_pre_trained_models, 'fwd_wt103_enc.h5')
fitos_file = os.path.join(directory_pre_trained_models, 'fitos_wt103.pkl')

In [110]:
wgts = torch.load(weights_file, map_location=lambda storage, loc: storage)

In [111]:
enc = torch.load(encoder_file, map_location=lambda storage, loc: storage)

In [115]:
for k,v in enc.items():
    print(f'{k}:    {v.size()}')

0.encoder.weight:    torch.Size([238462, 400])
0.encoder_with_dropout.embed.weight:    torch.Size([238462, 400])
0.rnns.0.module.weight_ih_l0:    torch.Size([4600, 400])
0.rnns.0.module.bias_ih_l0:    torch.Size([4600])
0.rnns.0.module.bias_hh_l0:    torch.Size([4600])
0.rnns.0.module.weight_hh_l0_raw:    torch.Size([4600, 1150])
0.rnns.1.module.weight_ih_l0:    torch.Size([4600, 1150])
0.rnns.1.module.bias_ih_l0:    torch.Size([4600])
0.rnns.1.module.bias_hh_l0:    torch.Size([4600])
0.rnns.1.module.weight_hh_l0_raw:    torch.Size([4600, 1150])
0.rnns.2.module.weight_ih_l0:    torch.Size([1600, 1150])
0.rnns.2.module.bias_ih_l0:    torch.Size([1600])
0.rnns.2.module.bias_hh_l0:    torch.Size([1600])
0.rnns.2.module.weight_hh_l0_raw:    torch.Size([1600, 400])
1.decoder.weight:    torch.Size([238462, 400])


In [92]:
# they're the same vals
wgts['rnns.2.module.bias_hh_l0'] == enc['0.rnns.2.module.bias_hh_l0']

tensor([1, 1, 1,  ..., 1, 1, 1], dtype=torch.uint8)

In [94]:
rnn_weights = {k:v for k,v in enc.items() if 'rnns' in k}

In [ ]:
vocab_size = 238462
embedding_size = 400
hidden_size = 4600
num_layers = 3

In [ ]:
# example of a model
lm = RNNLM(device, vocab_size, embedding_size, hidden_size, 256, num_layers=3)

In [119]:
?getattr

Docstring:
getattr(object, name[, default]) -> value

Get a named attribute from an object; getattr(x, 'y') is equivalent to x.y.
When a default argument is given, it is returned when the attribute doesn't
exist; without it, an exception is raised in that case.
Type:      builtin_function_or_method


In [120]:
for k, v in rnn_weights.items():
    layer_detail = k.split('.')
    print(layer_detail)
    # which rnn, and what layer???
    n_rnn = int(layer_detail[2])
    layer_name = layer_detail[-1].replace('_raw', '')
    layer = lm.rnns[n_rnn]
    try:
        # this is what assigns the new value
        getattr(layer, layer_name).data = v
    except Exception as e:
        print(e)

['0', 'rnns', '0', 'module', 'weight_ih_l0']
['0', 'rnns', '0', 'module', 'bias_ih_l0']
['0', 'rnns', '0', 'module', 'bias_hh_l0']
['0', 'rnns', '0', 'module', 'weight_hh_l0_raw']
['0', 'rnns', '1', 'module', 'weight_ih_l0']
['0', 'rnns', '1', 'module', 'bias_ih_l0']
['0', 'rnns', '1', 'module', 'bias_hh_l0']
['0', 'rnns', '1', 'module', 'weight_hh_l0_raw']
['0', 'rnns', '2', 'module', 'weight_ih_l0']
['0', 'rnns', '2', 'module', 'bias_ih_l0']
['0', 'rnns', '2', 'module', 'bias_hh_l0']
['0', 'rnns', '2', 'module', 'weight_hh_l0_raw']


In [121]:
lsmt0 = lm.rnns[0]

In [122]:
rnn_weights['0.rnns.0.module.weight_ih_l0']

tensor([[-0.0812, -0.0811, -0.0937,  ..., -0.0259, -0.1403, -0.3247],
        [ 0.1154,  0.1142,  0.0938,  ..., -0.0711,  0.1669, -0.0387],
        [-0.0051,  0.1007,  0.2071,  ..., -0.0860, -0.0288, -0.0894],
        ...,
        [ 0.0055,  0.0157,  0.2990,  ...,  0.0616,  0.1159, -0.4737],
        [ 0.0181,  0.0426,  0.1130,  ...,  0.3529, -0.0114, -0.0125],
        [-0.0167, -0.1328,  0.1741,  ...,  0.0548, -0.0045,  0.1688]])

In [123]:
lsmt0.weight_ih_l0.data

tensor([[-0.0812, -0.0811, -0.0937,  ..., -0.0259, -0.1403, -0.3247],
        [ 0.1154,  0.1142,  0.0938,  ..., -0.0711,  0.1669, -0.0387],
        [-0.0051,  0.1007,  0.2071,  ..., -0.0860, -0.0288, -0.0894],
        ...,
        [ 0.0055,  0.0157,  0.2990,  ...,  0.0616,  0.1159, -0.4737],
        [ 0.0181,  0.0426,  0.1130,  ...,  0.3529, -0.0114, -0.0125],
        [-0.0167, -0.1328,  0.1741,  ...,  0.0548, -0.0045,  0.1688]])